In [ ]:
# About the author
# Bhawesh Mishra
# 3rd year Undergraduate student at VIT, Vellore
# LinkedIn: https://www.linkedin.com/in/bhawesh-mishra-410398210/
# GitHub: https://github.com/BhaweshMishra

In [5]:
# Importing the modules
import cirq
import numpy as np

In [115]:
# Defining the function to generate a quantum gate matrix based on truth table
# Refer https://github.com/BhaweshMishra/Quantum-gate-matrix-generator

def truth_table_to_quantum_gate_matrix(fx):
      input_bits = len(list(fx.keys())[0])
      output_bits = len(fx[list(fx.keys())[0]])
      formated_string = '{:0'+str(input_bits+output_bits)+'b}'    #formatting for input bits
      formated_string_out = '{:0'+str(output_bits)+'b}'           #formatting for ouput bits  
      fx1 = {}                                                    #Dictionary to store new values including redundant qubits
      for i in range(0,2**(input_bits+output_bits)):
            inputstr = formated_string.format(i)
            outputstr1 = inputstr[0:input_bits]
            fxout = int(fx[inputstr[0:input_bits]],2)
            
            toxor = int(inputstr[input_bits:],2)
            xored = fxout^toxor
            xored_formated = formated_string_out.format(xored)
            outputstr = outputstr1 + xored_formated
            fx1[inputstr]=outputstr


      mat = np.zeros((2**(input_bits+output_bits),2**(input_bits+output_bits)))
      for i in fx1:
            matindex = int(i,2)
            matindexout = int(fx1[i],2)
            mat[matindex][matindexout] = 1


      mat = mat.transpose()
      return mat





In [116]:

# The truth table defined for the function
# Secret String (s) = 101
fx = {
    '000':'000',
    '001':'010',
    '010':'001',
    '011':'100',
    '100':'010',
    '101':'000',
    '110':'100',
    '111':'001',
    }


QGM = truth_table_to_quantum_gate_matrix(fx)
QGM
# QGM is the generated gate matrix

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 1., 0.]])

In [117]:
# Implementating the gate to use in the quantum circuit

class U(cirq.Gate):
    def __init__(self):
        super(U, self)

    def _num_qubits_(self):
        return 6

    def _unitary_(self):
        return QGM

    def _circuit_diagram_info_(self, args):
        return "u","u","u","u","u","u"

Uu = U()



In [118]:
# Building the quantum circuit

q = cirq.LineQubit.range(6)
mycirq = cirq.Circuit()
moment0 = cirq.Moment([cirq.H(q[i]) for i in range(6) if i <3])
moment1 = cirq.Moment([cirq.measure(q[i],key='a'+str(i)) for i in range(6) if i>2])
moment2 = [cirq.measure(q[i],key='b'+str(i)) for i in range(6) if i<3]
moment3 = [cirq.H(q[i]) for i in range(6) if i<3]
mycirq.append([moment0, Uu(*q), moment1, moment0, moment2])


mycirq

0: ───H───u─────────────H───M('b0')───
          │
1: ───H───u─────────────H───M('b1')───
          │
2: ───H───u─────────────H───M('b2')───
          │
3: ───────u───M('a3')─────────────────
          │
4: ───────u───M('a4')─────────────────
          │
5: ───────u───M('a5')─────────────────

In [119]:
# Simulating the results
sim = cirq.Simulator()
result = sim.run(mycirq,repetitions=10)
result

# You can manually check the result. The quantum circuit is run 10 times in this cell.
# In the real implementation, the quantum circuit does not need to be run 10 times.
#
# If the number of input qubits are n, the circuit needs to be run approximately n 
# times to obtain n simulataneous equations

a3=0010000101
a4=0000010000
a5=0000001000
b0=1010011101
b1=0100100010
b2=1010011101

In [129]:
# Since the unknown string is 3 bit, 3 simultaneous equations are required
no_of_equations = 3
eq_list = []
while (no_of_equations > 0):
    result = sim.run(mycirq,repetitions=1)
    result_dict = result.measurements
    np_arr = [result_dict['b0'][0][0],result_dict['b1'][0][0],result_dict['b2'][0][0]]
    if np_arr not in eq_list and np_arr!=[0,0,0]:
        eq_list.append(np_arr)
        no_of_equations -= 1
    

A = np.array((eq_list))
A

# The variable A contains the simulatneous equations extracted from the results from simulating the circuit

array([[1, 0, 1],
       [0, 1, 0],
       [1, 1, 1]], dtype=int8)

In [130]:
# Solving the equations simulateously

A = np.array((eq_list))
U, S, V = np.linalg.svd(A)
null_space = V[-1, :]

# Print the non-trivial solutions
# Value of the unknown string
s = abs(null_space.round())
# Secret string s
s


array([1., 0., 1.])

In [ ]:
# The value of s is the same as the value given in the start